Put MOSEK license file (`mosek.lic`) in 
`/root/mosek/mosek.lic` or `/user/mosek/mosek.lic`

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import cm

mpl.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 14
plt.rcParams['axes.linewidth'] = 2
colors = cm.get_cmap('tab10', 5)

In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
import numpy as np
import pandas as pd
import cvxpy as cp
import yfinance as yf
import pypfopt
import pathlib
import sklearn
import time
from tqdm.notebook import tqdm
from pathlib import Path
from dateutil.relativedelta import relativedelta
from sklearn.linear_model import LinearRegression
print(pypfopt.__version__)
print(cp.installed_solvers())

1.4.1
['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'GUROBI', 'MOSEK', 'OSQP', 'SCS']


In [4]:
from pypfopt.risk_models import sample_cov, CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.expected_returns import mean_historical_return, capm_return

In [5]:
print(f"Working Directory: {Path.cwd()}")
source = Path.cwd()
p = source.glob('*.csv')
files = [f for f in p if f.is_file()]
files

Working Directory: C:\Users\ashry\OneDrive - National University of Singapore\NUS\Year 4\ST4199 Final Year Project in Statistics\st4199\data


[WindowsPath('C:/Users/ashry/OneDrive - National University of Singapore/NUS/Year 4/ST4199 Final Year Project in Statistics/st4199/data/Backtest-daily-ALL.png'),
 WindowsPath('C:/Users/ashry/OneDrive - National University of Singapore/NUS/Year 4/ST4199 Final Year Project in Statistics/st4199/data/Backtest-daily-final.png'),
 WindowsPath('C:/Users/ashry/OneDrive - National University of Singapore/NUS/Year 4/ST4199 Final Year Project in Statistics/st4199/data/Backtest-daily-no-trading-costs.png'),
 WindowsPath('C:/Users/ashry/OneDrive - National University of Singapore/NUS/Year 4/ST4199 Final Year Project in Statistics/st4199/data/Backtest-daily-trading-costs.png'),
 WindowsPath('C:/Users/ashry/OneDrive - National University of Singapore/NUS/Year 4/ST4199 Final Year Project in Statistics/st4199/data/BAD_backtest (market impact).ipynb'),
 WindowsPath('C:/Users/ashry/OneDrive - National University of Singapore/NUS/Year 4/ST4199 Final Year Project in Statistics/st4199/data/crsp.xlsx'),
 Win

In [6]:
pd.set_option('display.max_columns', 99)
%time crsp = pd.read_excel(source / "crsp.xlsx", skiprows=1, sheet_name = 'Sheet1', engine='openpyxl')
print(crsp.shape)
%time msci = pd.read_excel(source / "msci.xlsx", skiprows=1, sheet_name="Sheet1", engine="openpyxl")
print(msci.shape)
%time tickers_data = pd.read_csv(source / "TICKERS.csv", low_memory=False)
sectors = tickers_data[['ticker', 'name', 'sector', 'industry', 'scalemarketcap']].drop_duplicates()
print(sectors.shape)
%time sep = pd.read_csv(source / "SEP.csv", parse_dates=['date'])
print(sep.shape)
%time daily = pd.read_csv(source / "DAILY.csv", parse_dates=['date'])
print(daily.shape)
daily.head()

Wall time: 634 ms
(2435, 2)
Wall time: 369 ms
(5215, 2)
Wall time: 259 ms
(29326, 5)
Wall time: 48.8 s
(37383175, 10)
Wall time: 33.7 s
(31666308, 10)


,ticker,date,lastupdated,ev,evebit,evebitda,marketcap,pb,pe,ps
0,A,2018-10-19,2018-10-21,20219.1,20.1,16.7,20551.1,4.5,69.0,4.3
1,AA,2018-10-19,2018-10-21,8037.9,8.4,4.6,7197.9,1.4,50.7,0.6
2,AAL,2018-10-19,2018-10-21,38354.7,10.9,6.9,14754.7,-17.0,9.0,0.3
3,AAMC,2018-10-19,2018-10-21,49.8,-5.7,-6.1,79.1,-0.4,-8.3,4.8
4,AAME,2018-10-19,2018-10-21,83.0,27.1,19.6,56.7,0.6,53.9,0.3


In [7]:
def one_year_prices(df, tickers, t):
    '''
    Returns a pd.Series of the one-year Daily Average Close Price of given ticker(s) from time t
    
    Parameters:
        df::DataFrame: SEP.csv, with a 'date' DateTime64 column or similar
        tickers::[Strings]: List of strings of names of ticker(s) to get the previous year DAV
        t::pd.Timestamp or similar: Date to filter on
        timedelta::str: 'd, day, days' or 'y, year, years' get DAV of previous day/year
    
    Returns:
        dav::float: average one-year daily volume
    '''
    timedelta = relativedelta(years=1)
    # get range of prev_period to t
    prev_period = t - timedelta
    
    if t not in df['date'].values:
        return 'No data for {t}'.format(t=t)
    
    if not df['ticker'].isin(tickers).any():
        return 'No ticker for any of {tickers}'.format(tickers="tickers")

    if isinstance(df.index, pd.DatetimeIndex):
        mask = (df['ticker'].isin(tickers)) & \
               (df.loc[df.index >= prev_period]) & \
               (df.loc[df.index < t])
    else:
        mask = (df['ticker'].isin(tickers)) & \
               (df['date'] >= prev_period) & \
               (df['date'] < t)
    
    temp_df = df[mask]
    prices = temp_df.groupby('ticker').mean()['close']
    return prices

In [8]:
small_tickers = ["MSFT", "AMZN", "KO", "MA", "COST", 
           "LUV", "XOM", "PFE", "JPM", "UNH", 
           "ACN", "DIS", "GILD", "F", "TSLA"] 
t = pd.Timestamp('2020-05-28')
%time prices = one_year_prices(sep, small_tickers, t)
prices

Wall time: 3.8 s


ticker
ACN      190.335850
AMZN    1921.511067
COST     292.010455
DIS      130.662605
F          8.230553
GILD      68.144585
JPM      115.285217
KO        52.253123
LUV       48.769249
MA       280.059486
MSFT     151.662273
PFE       37.703814
TSLA     425.757905
UNH      261.062055
XOM       63.058063
Name: close, dtype: float64

In [9]:
start_date = ( t-relativedelta(years=1) ).strftime('%Y-%m-%d')
end_date = t.strftime('%Y-%m-%d')
print(start_date, end_date)
ohlc = yf.download(tickers = small_tickers, start=start_date, end=end_date, interval='1d')
prices = ohlc['Adj Close']
prices.mean(axis=0)

2019-05-28 2020-05-28
[*********************100%***********************]  15 of 15 completed


ACN      186.730557
AMZN    1921.507350
COST     281.440911
DIS      130.120911
F          8.044081
GILD      64.376162
JPM      109.825472
KO        49.865217
LUV       48.473459
MA       277.922281
MSFT     149.541643
PFE       33.905699
TSLA      85.151581
UNH      255.277696
XOM       57.100639
dtype: float64

In [10]:
# filter out daily DAV of t-1
def one_year_DAV(df, tickers, t):
    '''
    Returns a pd.Series of the one-year Daily Average Volume of given ticker(s) from time t
    
    Parameters:
        df::DataFrame: SEP.csv, with a 'date' DateTime64 column or similar
        tickers::[Strings]: List of strings of names of ticker(s) to get the previous year DAV
        t::pd.Timestamp or similar: Date to filter on
        timedelta::str: 'd, day, days' or 'y, year, years' get DAV of previous day/year
    
    Returns:
        dav::float: average one-year daily volume
    '''
    timedelta = relativedelta(years=1)
    # get range of prev_period to t
    prev_period = t - timedelta
    
    if t not in df['date'].values:
        return 'No data for {t}'.format(t=t)
    
    if not df['ticker'].isin(tickers).any():
        return 'No ticker for any of {tickers}'.format(tickers="tickers")

    if isinstance(df.index, pd.DatetimeIndex):
        mask = (df['ticker'].isin(tickers)) & \
               (df.loc[df.index >= prev_period]) & \
               (df.loc[df.index < t])
    else:
        mask = (df['ticker'].isin(tickers)) & \
               (df['date'] >= prev_period) & \
               (df['date'] < t)
    
    temp_df = df[mask]
    dav = temp_df.groupby('ticker').mean()['volume']
    return dav

In [11]:
def trading_limit(davs, max_fraction=0.01):
    '''
    Return a the limit of trade constraints
    Parameters:
          df::pd.DataFrame: SEP.csv, with a 'date' DateTime64 column or similar
          tickers::[str] List of strings of names of ticker(s) to get previous year DAV
          t::pd.Timestamp or similar: date to filter DAVS on
          max_fraction::float: to specify trading limit x% of DAV
          
    Returns:
        np.darray of trade limits
        e.g. [255, 12, ..., 3]
    '''
    #davs = one_year_DAV(df, tickers, t)
    w_constraint = np.array( davs * max_fraction )
    return w_constraint

In [12]:
from pypfopt import EfficientFrontier, objective_functions
from pypfopt import expected_returns, risk_models, EfficientFrontier

mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)

ef_no_constraints = EfficientFrontier(mu, S, solver="SCS")
ef_no_constraints.min_volatility()
ef_no_constraints_weights = ef_no_constraints.weights

In [13]:
nav = 100000000 # 100 million
max_fraction = 0.01
w = ef_no_constraints_weights
latest_prices = prices.mean(axis=0)
latest_davs = one_year_DAV(sep, small_tickers, t)

In [14]:
ef_no_constraints.clean_weights()

OrderedDict([('ACN', 0.0),
             ('AMZN', 0.24787),
             ('COST', 0.2876),
             ('DIS', 0.0),
             ('F', 0.0384),
             ('GILD', 0.15384),
             ('JPM', 0.0),
             ('KO', 0.15829),
             ('LUV', 0.0),
             ('MA', 0.0),
             ('MSFT', 0.0),
             ('PFE', 0.114),
             ('TSLA', 0.0),
             ('UNH', 0.0),
             ('XOM', 0.0)])

In [15]:
ef_constraints = EfficientFrontier(mu, S, weight_bounds=(-1,1), solver="MOSEK", verbose=False)
ef_constraints.add_constraint( lambda w : cp.abs(w) * nav / latest_prices <= trading_limit(latest_davs, 0.01) )
ef_constraints.min_volatility()
ef_constraints_weights = ef_constraints.weights
ef_constraints.clean_weights()

OrderedDict([('ACN', 0.04211),
             ('AMZN', 0.71005),
             ('COST', 0.07107),
             ('DIS', 0.12449),
             ('F', 0.04548),
             ('GILD', 0.07584),
             ('JPM', -0.07453),
             ('KO', 0.07273),
             ('LUV', 0.03724),
             ('MA', -0.0176),
             ('MSFT', -0.3183),
             ('PFE', 0.08369),
             ('TSLA', -0.0277),
             ('UNH', 0.06344),
             ('XOM', 0.11201)])

In [16]:
b = cp.Variable(len(mu), boolean=True)
mins = np.repeat(0.5/100, len(mu)) # 0.50% threshold
maxs = np.repeat(2/100, len(mu))

ef_all_constraints = EfficientFrontier(mu, S, weight_bounds=(-1,1), solver='MOSEK', verbose=False) 
ef_all_constraints.add_constraint(lambda x: x >= cp.multiply(b, mins)) 
ef_all_constraints.min_volatility() 
ef_all_constraints_weights = ef_all_constraints.clean_weights() 
ef_all_constraints_weights 

OrderedDict([('ACN', 0.0),
             ('AMZN', 0.24788),
             ('COST', 0.28758),
             ('DIS', 0.0),
             ('F', 0.0384),
             ('GILD', 0.15383),
             ('JPM', 0.0),
             ('KO', 0.15833),
             ('LUV', 0.0),
             ('MA', 0.0),
             ('MSFT', 0.0),
             ('PFE', 0.11397),
             ('TSLA', 0.0),
             ('UNH', 0.0),
             ('XOM', 0.0)])

# Market Impact

In [17]:
def market_impact(daily, sep, benchmark, vix, list_of_tickers, currdf, prevdf, t):
    """
        Returns the market impact of a trade on the market

            Parameters:
                daily::pd.DataFrame: DAILY.csv
                sep::pd.DataFrame: SEP.csv
                benchmark::pd.DataFrame: mscius.xlsx ('Exchange Date', 'Close')
                crsp::pd.DataFrame: crsp.xlsx ('Exchange Date', Close')
                ticker::[Strings]: List of strings of names of ticker(s)
                t::pd.Timestamp: Date of rebalance
                currdf::DataFrame: Current DataFrame with a 'long'(bool), 'price', 'shares' column
                prevdf::DataFrame: Previous DataFrame with a 'long'(bool), 'price', 'shares' column 


            Returns:
                market_impact::float: Estimated Market Impact of the trade   

        Market Impact Model uses estimated coefficients from Table VII of Frazzini et al - Trading Costs (2018)
        Column (5) for a Global sample, Column (10) for US sample, Column (15) for International

        MI = a + b*x + c*sign(x)*sqrt( abs(x) )
        x = m/dtv*100%, where m = sign(dollar volume) and dtv =  stock's average one year dollar volume
        dollar volume: total value of shares traded = volume * price
        
    """
    #print(list_of_tickers)
    # Theta_2: Time trend
    theta_2 = -0.01
    
    start_time = pd.Timestamp('1926-06-01')
    curr_time = pd.Timestamp(t)
    if curr_time.month >= start_time.month:
        diff_in_years = curr_time.year - start_time.year
    else:
        diff_in_years = curr_time.year - 1 - start_time.year
    
    theta2 = theta_2 * diff_in_years
    
    # Theta_3: Log (1 + market cap)
    theta_3 = -0.14
    timedelta = relativedelta(days=1)
    # get range of prev_period to t
    prev_period = t - timedelta
    
    if prev_period not in daily['date'].values:
        #print(prev_period, 'not available')
        # This is a catch for long weekends/holidays where there is no available data
        for i in range(4):
            prev_period = prev_period - relativedelta(days = 1)
            if prev_period in daily['date'].values:
                print("Theta_3: found data for prev_period:", prev_period)
                break
    #print('\nTheta_3 new prev_period:', prev_period)
    
    if ~daily['ticker'].isin(list_of_tickers).any():
        return 'Theta_3: No ticker for any of {tickers}'.format(tickers="tickers")
       
    mask = (daily['ticker'].isin(list_of_tickers)) & (daily['date'] == prev_period)
    
    temp_df = daily[mask].drop_duplicates()
    market_value_of_equity = pd.Series(index = temp_df['ticker'], 
                           data = (temp_df[['marketcap']].values/1000).flatten(), 
                           name = 'marketcap', dtype='float64')
    market_value_of_equity.fillna(1e-5, inplace=True)
    #print("market_cap\n", market_value_of_equity)
    tickers_series = pd.Series(data=list_of_tickers, name='tickers')
    #print("tickers\n", tickers_series)    
    temp = pd.merge(market_value_of_equity, tickers_series, left_index=True, right_on='tickers', how='inner').drop_duplicates()
    market_value_of_equity = pd.Series(data = temp['marketcap'].values, index=temp['tickers'], name='market_value_of_equity', dtype='float64')
    
    if len(market_value_of_equity) < len(list_of_tickers) and len( set(list_of_tickers) ^ set(market_value_of_equity.index) ) > 0:
        print("Theta_3: {tickers} data not available in DAILY.csv".format(tickers = set(list_of_tickers) ^ set(market_value_of_equity.index)))
 
        missing_tickers = list(set(list_of_tickers) ^ set(market_value_of_equity.index))
        marketcap_missing_tickers = []
        # Download the latest market cap from Yahoo Finance
        for ticker in missing_tickers:
            marketcap_missing_tickers.append(yf.Ticker(ticker).info['marketCap'] / 1000000000)

        missing_data = pd.Series(data=marketcap_missing_tickers, index=missing_tickers, name='marketcap_missing_tickers', dtype='float64')
        market_value_of_equity = pd.concat([market_value_of_equity, missing_data], axis=0) 

    try:
        ans = np.log(1 + market_value_of_equity)
    except TypeError:
        print('\n')
        print("Theta_3: Check market_cap function returns a float")
    theta3 = ans * theta_3
    
    # Theta_4 and Theta_5: Fraction of Daily Volume and Sqrt(Fraction of Daily Volume)
    theta_4, theta_5 = -0.53, 11.21
    
    timedelta = relativedelta(years=1)
    # get range of prev_period to t
    try:
        prev_period = t - timedelta
    except:
        print("Theta_4_5: t is not a Datetime", t)
    #print(list_of_tickers)
    mask = (sep['ticker'].isin(list_of_tickers)) & (sep['date'] == prev_period)
    if sep[mask].shape[0] < 1:
        #print('Theta_4_5_6:No data for {t} in Data'.format(t=prev_period))
        for i in range(4):
            prev_period = prev_period - relativedelta(days = 1)
            mask = (sep['date'] == prev_period) & (sep['ticker'].isin(list_of_tickers))
            if sep[mask].shape[0] == len(list_of_tickers):
                print("Theta_4_5_6: found new prev_period with all data:", prev_period)
                break
        
    print('\nNew prev_period', prev_period, 't', t)

    if t not in sep['date'].values:
        return 'Theta_4, Theta_5:No data for {t}'.format(t=t)
    
    if not sep['ticker'].isin(list_of_tickers).any():
        return 'Theta_4, Theta_5: No ticker for any of {tickers}'.format(tickers="tickers")
    
    mask = (sep['ticker'].isin(list_of_tickers)) & \
           (sep['date'] >= prev_period) & \
           (sep['date'] < t)
    
    temp_df = sep[mask]
    
    # Theta_6: Idiosyncratic Volatility
    # uses the same dataframe as Theta_4 and Theta_5 so I put it here for efficiency
    theta_6 = 0.31
    if temp_df.shape[0] < 1:
        return 'Theta_6: No data for {t} in data'.format(t=t)
    prices_df = temp_df.sort_values('date', axis=0, ascending=True, ignore_index=True)
    prices_df['returns'] = prices_df.groupby(['ticker'])['close'].pct_change()
    returns_df = prices_df[['date', 'ticker', 'returns']].set_index('date').fillna(0)

    mask = (benchmark.iloc[:,0] >= prev_period) & (benchmark.iloc[:,0] < t)
    if benchmark[mask].shape[0] < 1:
        return 'Theta_6: No data for {t} in data'.format(t=t)
    benchmark_df = benchmark[mask].sort_values('Exchange Date', axis=0, ascending=True, ignore_index=True).set_index('Exchange Date')
    benchmark_df = benchmark_df['Close'].pct_change().fillna(0) # Close is now returns
    volatility_df = pd.merge(returns_df, benchmark_df, how='inner', left_index=True, right_index=True, validate='m:1')
  
    idiosyncratic_volatility_list = []
    for ticker in list_of_tickers:
        mask = volatility_df['ticker'] == ticker
        X = volatility_df[mask]['returns'].values.reshape(-1,1)
        y = volatility_df[mask]['Close'].values
        model = LinearRegression()
        model.fit(X, y)
        preds = model.predict(X)
        residuals = preds - y
        idiosyncratic_volatility_list.append( np.std(residuals, ddof=1) )

    idiosyncratic_volatility = pd.Series(data=idiosyncratic_volatility_list, index=list_of_tickers, name="idiosyncratic_volatility", dtype='float64')
    idiosyncratic_volatility.drop_duplicates(inplace=True)
    theta6 = theta_6 * idiosyncratic_volatility
    
    # Theta_4 and Theta_5: Fraction of Trade of 1-year Daily Average Volume
    
    davs = temp_df.groupby('ticker').mean()['volume']

    temp_df = pd.merge(davs, prevdf, left_index=True, right_on='ticker', how='inner')
    temp_df = pd.merge(temp_df, currdf.set_index('ticker')[['long', 'shares', 'price']], left_on='ticker', right_index=True,
                      how='inner', suffixes=("_prev", "_curr"))
    #temp_df.drop_duplicates(inplace=True) # not sure if this affects anything if error remove this comment

    size_of_trades = np.zeros(len(temp_df))
    
    for i in range(len(temp_df)):
        new_shares = temp_df.iloc[i]['shares_curr'] - temp_df.iloc[i]['shares_prev']
        size_of_trades[i] = new_shares * temp_df.iloc[i]['price_curr']

    size_of_trade = pd.Series(data = size_of_trades, index = temp_df['ticker'], name = 'size_of_trade', dtype='float64')

    temp_df = pd.merge(temp_df, size_of_trade, left_on='ticker', right_index=True, how='inner')
    temp_df['frac_of_daily_volume'] = np.where(temp_df['size_of_trade'] != 0,  temp_df['size_of_trade'] / temp_df['volume'], 0)
    temp_df['sqrt_of_frac_of_daily_volume'] = np.sqrt(np.abs(temp_df['frac_of_daily_volume']))
    
    temp_df.sort_index(inplace=True)
    temp_df.set_index('ticker', inplace=True)
    theta4 = theta_4 * temp_df['frac_of_daily_volume'] 
    theta5 = theta_5 * temp_df['sqrt_of_frac_of_daily_volume']
    
    # Theta_7
    theta_7 = 0.12
    
    prev_period = t - relativedelta(months=1)
    mask = (vix.iloc[:,0] >= prev_period) & (vix.iloc[:,0] < t)
    if vix[mask].shape[0] < 1:
        return 'No data for {t}'.format(t=t)
    vix_values = vix[mask].iloc[:,1].pct_change().fillna(0)
    
    theta7 = theta_7 * vix_values
    
    a = theta2 + np.median(theta3) + np.median(theta6) + np.median(theta7)
    #a = np.median(theta3) + np.median(theta6) + np.median(theta7)
    b = theta4
    c = theta5
    marketimpact = a + b + c
    #print('\ntheta_2', theta2, '\ntheta3\n', theta3, '\ntheta4\n', theta4, '\ntheta5\n', theta5, '\ntheta6\n', theta6, '\ntheta7\n', theta7, '\n')
    return marketimpact

# Ensemble Portfolio

In [18]:
def get_dataframe(df, quarter='2017-1'):
    '''
    Filters the predictions and ground truth
    from the ensemble for specific quarter
    
    Parameters:
        df::pd.DataFrame: testing_predictions.csv or preds
        quarter::str: in the format: YYYY-Q
        2017-1 refers to 2017, Q1 predictions 
    Returns: 
        DataFrame
    '''
    mask = df['year-quarter'].values == quarter
    temp_df = df[mask].reset_index(drop=True)
    temp_df.rename(columns={"ground_truth": "y", "ensemble_prediction": "y_pred"}, inplace=True)
    return temp_df

In [19]:
def get_prices(df, list_of_tickers, prev_t, curr_t):
    '''
    Returns a pd.DataFrames of the latest Closing Price of given ticker(s) from between prev_t and curr_t 

    Parameters:
        df::DataFrame: SEP.csv, with a 'date' DateTime64 column or similar
        list_of_tickers::[Strings]: List of strings of names of ticker(s) to get the previous year DAV
        prev_t::pd.Timestamp: Prices more recent than `prev_t` to filter on 
        curr_t::pd.Timestamp: Prices older than `curr_t` to filter on
    
    Returns:
        prices::pd.DataFrame: Index are the dates, columns are the tickers, and values are the Adjusted Close Price
    '''
    # get range of prev_period to t
    t = curr_t
    prev_period = prev_t
    
    if prev_period not in df['date'].values:
        #print(prev_period, 'not available')
        # This is a catch for long weekends/holidays where there is no available data
        for i in range(4):
            prev_period = prev_period - relativedelta(days = 1)
            if prev_period in df['date'].values:
                print("Theta_3: found data for prev_period:", prev_period)
                break
    #print('\nTheta_3 new prev_period:', prev_period)
    
    if ~df['ticker'].isin(list_of_tickers).any():
        return 'Theta_3: No ticker for any of {tickers}'.format(tickers="tickers")
       
    mask = (df['ticker'].isin(list_of_tickers)) & (df['date'] >= prev_period) & (df['date'] <= t)
    
    temp_df = df[mask].drop_duplicates()
    
    prices = pd.pivot(temp_df, index='date', columns='ticker', values='close')
    print('Requesting for prices at time', t)
    print('Returning ex-ante prices at time', t, '\nfrom', prev_period, 'to', t)
    
    # return temp_df # uncomment this to debug
    return prices

In [20]:
def get_latest_prices(df, list_of_tickers, t):
    '''
    Returns a pd.Series of the latest Closing Price of given ticker(s) from time t
    
    Parameters:
        df::DataFrame: SEP.csv, with a 'date' DateTime64 column or similar
        list_of_tickers::[Strings]: List of strings of names of ticker(s) to get the previous year DAV
        t::pd.Timestamp or similar: Date to filter on
    
    Returns:
        latest_prices::float: , Index are the tickers, values are the Latest Adjusted Close price
    '''
    timedelta = relativedelta(days=1)
    # get range of prev_period to t
    prev_period = t - timedelta
    
    if prev_period not in df['date'].values:
        #print(prev_period, 'not available')
        # This is a catch for long weekends/holidays where there is no available data
        for i in range(4):
            prev_period = prev_period - relativedelta(days = 1)
            if prev_period in df['date'].values:
                print("Theta_3: found data for prev_period:", prev_period)
                break
    #print('\nTheta_3 new prev_period:', prev_period)
    
    if ~df['ticker'].isin(list_of_tickers).any():
        return 'Theta_3: No ticker for any of {tickers}'.format(tickers="tickers")
       
    mask = (df['ticker'].isin(list_of_tickers)) & (df['date'] == prev_period)
    
    temp_df = df[mask].drop_duplicates()
    
    latest_prices = temp_df.set_index('ticker')['close']
    print('Requesting for prices at time', t)
    print('Returning ex-ante prices at time', prev_period)
    
    #return temp_df uncomment this to debug
    return latest_prices

In [21]:
def get_one_year_prices(df, list_of_tickers, t):
    '''
    Returns a pd.DataFrame of the latest Closing Price of given ticker(s) from 1_year before time t
    
    Index are the dates, columns are the tickers, and values are the Adjusted Close Price

    
    Parameters:
        df::DataFrame: SEP.csv, with a 'date' DateTime64 column or similar
        list_of_tickers::[Strings]: List of strings of names of ticker(s) to get the previous year DAV
        t::pd.Timestamp or similar: Date to filter on
    
    Returns:
        prices::pd.DataFrame: Index are the dates, columns are the tickers, and values are the Adjusted Close Price
    '''
    timedelta = relativedelta(years=1)
    # get range of prev_period to t
    prev_period = t - timedelta
    
    if prev_period not in df['date'].values:
        #print(prev_period, 'not available')
        # This is a catch for long weekends/holidays where there is no available data
        for i in range(4):
            prev_period = prev_period - relativedelta(days = 1)
            if prev_period in df['date'].values:
                print("Theta_3: found data for prev_period:", prev_period)
                break
    #print('\nTheta_3 new prev_period:', prev_period)
    
    if ~df['ticker'].isin(list_of_tickers).any():
        return 'Theta_3: No ticker for any of {tickers}'.format(tickers="tickers")
       
    mask = (df['ticker'].isin(list_of_tickers)) & (df['date'] >= prev_period) & (df['date'] < t)
    
    temp_df = df[mask].drop_duplicates()
    
    one_year_prices = pd.pivot(temp_df, index='date', columns='ticker', values='close')
    print('Requesting for prices at time', t)
    print('Returning ex-ante prices at time', prev_period)
    
    # return temp_df # uncomment this to debug
    return one_year_prices

In [22]:
def get_predictions(prev_t, curr_t_str, method='ledoit_wolf'):
    '''
    Returns ensemble_mu and S, given curr_t and prev_t

    e.g. to Rebalance Q4-2016 portfolio using Q1-2017 preds,
    prev_t = 2016-q4, curr_t = "2017-1"

    Parameters:
        prev_t::pd.Timestamp or similar:
        curr_t_str::str: in the format YYYY-Q, 
            i.e. 2017-Q1  is '2017-1
        method::str: 'ledoit_wolf' or 'sample_cov' for risk model
    '''
    predictions = get_dataframe(preds, curr_t_str)
    ensemble_mu = predictions.set_index('ticker')['y_pred']
    
    list_of_tickers = predictions['ticker']

    curr_t_idx = timestamps.index(prev_t) + 1
    curr_t = timestamps[curr_t_idx]
    #print(curr_t)
    #prices = get_prices(sep, list_of_tickers, prev_t, curr_t-relativedelta(days=1))
    prices = get_prices(sep, list_of_tickers, prev_t-relativedelta(months=3), prev_t)
    if method == 'ledoit_wolf':
        S = CovarianceShrinkage(prices).ledoit_wolf()
    elif method == 'sample_cov':
        S = pypfopt.risk_models.sample_cov(prices)
    #print(S.shape)
    ensemble_mu = predictions.set_index('ticker')['y_pred']
    #print(ensemble_mu.shape)
    
    return ensemble_mu, S

In [23]:
def clean_weights(weights, cutoff=1e-4, rounding=5):
    """
    Helper method to clean the raw weights, setting any weights whose absolute
    values are below the cutoff to zero, and rounding the rest.

    :param cutoff: the lower bound, defaults to 1e-4
    :type cutoff: float, optional
    :param rounding: number of decimal places to round the weights, defaults to 5.
                        Set to None if rounding is not desired.
    :type rounding: int, optional
    :return: asset weights
    :rtype: np.array
    """
    if weights is None:
        raise AttributeError("Weights not yet computed")
    clean_weights = weights.copy()
    clean_weights[np.abs(clean_weights) < cutoff] = 0
    if rounding is not None:
        if not isinstance(rounding, int) or rounding < 1:
            raise ValueError("rounding must be a positive integer")
        clean_weights = np.round(clean_weights, rounding)

    return clean_weights

In [24]:
def allocate(budget, weights, prices):
    '''
    Given current budget, allocate assets based on 100 collateral
    
    Parameters:
        budget::float: current budget available to buy stocks
        weights::np.array: of weight
        price::pd.Series: of latest price, ex-ante

    Returns:
        shares::np.array: of shares
    '''
    w = weights.copy()
    #print(w)
    if isinstance(prices, pd.Series):
        price = prices.values.copy()
    else:
        price = prices.copy()
    allocation = 1/np.sum(np.abs(w)) * budget * w
    #print(len(allocation))
    #print(price)
    shares = allocation // price
    return np.array(shares)

In [25]:
def initial_pf(df, pred, quarter, t, weights=None, start_capital=100000000):
    '''
    Returns a DataFrame in a suitable format for
    to initialize the initial portfolio and pick out EW L/S
    
    Parameters:
        df::pd.DataFrame: SEP.csv with a 'date' Datetime64 column
        pred::pd.DataFrame: testing_predictions.csv with a 'year-quarter' column
        quarter::str: in the format: YYYY-Q, 2017-1 refers to 2017, Q1 predictions 
        t::pd.Timestamp: pd.Timestamp equivalent of previous quarter so 2016-4 for Q1-2017 preds
        w::np.array: array of weights, if not provided defaults to EW L/S
        start_capital::int or float: starting capital, defaults to 100 Million
        
    Returns:
        pd.DataFrame: with the following columns:
            'decile' column (10 is top decile, 1 is bottom decile)
            'long' column (1 is Long, -1 is Short, 0 is nothing)
            'price' column (Current Close prices of tickers at time t)
    '''
    temp_preds = get_dataframe(preds, quarter)
    mask = df['date'] == t
    temp_prices = df[mask].set_index('ticker')['close']
    temp_prices.name = 'price'
    temp_df = pd.merge(temp_preds, temp_prices, left_on = 'ticker', right_index=True).reset_index(drop=True)
    
    if temp_df.shape[0] != temp_preds.shape[0] and len( set(temp_prices.index) ^ set(temp_preds['ticker']) ) > 0:
        print('{tickers} do not have price'.format( tickers=set(temp_prices.index) ^ set(temp_preds['ticker']) ))
    
    if weights is None:
        temp_df['decile'] = pd.qcut(temp_df['y_pred'], 10, labels=np.arange(1,11))
        conditions = [
            temp_df['decile'] == 10,
            temp_df['decile'] == 1
        ]
        values = [1, -1]
        temp_df['long'] = np.select(conditions, values, default=0)
        mask = temp_df['long'] == 0
        number_of_long_short = temp_df[~mask].shape[0]
        temp_df['weights'] = np.where(temp_df['long'] == 0, 0, temp_df['long']/number_of_long_short)
        temp_df.drop(['decile'], axis=1, inplace=True)

    else:
        temp_df['weights'] = weights
        conditions = [
                      temp_df['weights'] > 0,
                      temp_df['weights'] < 0
        ]
        values = [1, -1]
        temp_df['long'] = np.select(conditions, values, default=0)

    '''
    uniques, counts = np.unique(temp_df['long'], return_counts=True)
    aggregated_positions_dict = dict(zip(uniques, counts))
    number_of_long_stocks.append(aggregated_positions_dict[1])
    number_of_short_stocks.append(aggregated_positions_dict[-1])
    '''

    temp_df['shares'] = allocate(start_capital, temp_df['weights'].values, temp_df['price'].values)
    temp_df['asset_value'] = temp_df['shares'] * temp_df['price']
    temp_df['time'] = t

    return temp_df.drop(['y', 'y_pred', 'year-quarter'], axis=1)

In [26]:
def _objective_value(w, obj):
    """
    Helper method to return either the value of the objective function
    or the objective function as a cvxpy object depending on whether
    w is a cvxpy variable or np array.

    :param w: weights
    :type w: np.ndarray OR cp.Variable
    :param obj: objective function expression
    :type obj: cp.Expression
    :return: value of the objective function OR objective function expression
    :rtype: float OR cp.Expression
    """
    if isinstance(w, np.ndarray):
        if np.isscalar(obj):
            return obj
        elif np.isscalar(obj.value):
            return obj.value
        else:
            return obj.value.item()
    else:
        return obj

def market_impact_objective(w, score):
    '''
    minimize dot product of weight and score (Market Impact)

    Parameters:
        w::np.darray or cp.Variable:weights
        score::np.darray: market_impact (bps)
    Returns:
        float or CP.Expression: Market Impact
    '''
    market_impact_obj = w @ score
    return _objective_value(w, market_impact_obj)


def TE_constraint(w, w_benchmark, cov_matrix):
    '''
    Returns the squared of the ex-ante tracking error

    Parameters:
        w::np.darray or cp.Variable: weights
        w_benchmark::np.darray or cp.Variable: weight of benchmarks (weight + market_impact score)
        cov_matrix:: np.darray: covariance matrix
    Return:
        float or CP.Expression: tracking error
    '''    
    #print(w.shape, w_benchmark.shape)
    relative_weights = w-w_benchmark
    tracking_error = cp.quad_form(relative_weights, cov_matrix)
    return _objective_value(w, tracking_error)

In [59]:
def rebalance_unconstrainted(df, curr_t, method='ledoit_wolf'):
    '''
    Rebalances unconstrainted Mean-Variance Portfolio at curr_t
    
    df::pd.DataFrame: temp_mv_no_constraints_pf
    curr_t::pd.Timestamp: time of rebalancing
    
    curr_t must be q1_2017 onwards
    '''
    prev_period_preds_idx = timestamps.index(curr_t) - 1
    next_period_preds_idx = timestamps.index(curr_t) + 1
    prev_t_str = timestamps_str[prev_period_preds_idx]
    prev_t = timestamps[prev_period_preds_idx]
    next_t_str = timestamps_str[next_period_preds_idx]
    next_t = timestamps[next_period_preds_idx]
    prev_year = curr_t - relativedelta(years=1)
    print(prev_t, '\t', curr_t, '\t', next_t_str)
    
    predictions = get_dataframe(preds, next_t_str)
    ensemble_mu = predictions.set_index('ticker')['y_pred']
    actual_mu = predictions.set_index('ticker')['y']
    
    list_of_tickers = predictions['ticker'].values
    prices = get_prices(sep, list_of_tickers, prev_t, curr_t)
    latest_prices = prices.iloc[-2].values
    if method == 'ledoit_wolf':
        S = CovarianceShrinkage(prices).ledoit_wolf()
    elif method == 'sample_cov':
        S = pypfopt.risk_models.sample_cov(prices)

    cov_matrix = pypfopt.risk_models.risk_matrix(prices, returns_data=False, method='sample_cov')
    print('\n')
    print('-----------------------------------------------------------------------')
    print(f"Unconstrainted Mean-Variance Optimization for rebalancing at {curr_t}")
    start_time = time.time()
    ef_no_constraints = EfficientFrontier(ensemble_mu, S, weight_bounds=(-1,1), solver="MOSEK")
    ef_no_constraints.efficient_risk(0.05)
    ef_no_constraints_weights = clean_weights(ef_no_constraints.weights, cutoff=0.005)
    mv_no_constraints_weights.append(ef_no_constraints_weights)
    print(f"Optimization runtime: {time.time() - start_time:.3f} second(s)")
    
    ex_ante_mv_no_constraints_pf = df.drop(['price_curr'], axis=1)
    ex_ante_mv_no_constraints_pf.rename(columns={'price_prev':'price'}, inplace=True)
    ex_ante_mv_no_constraints_pf['price'] = latest_prices
    ex_ante_mv_no_constraints_pf['time'] = curr_t
    ex_ante_mv_no_constraints_pf['asset_value'] = ex_ante_mv_no_constraints_pf['shares'] * ex_ante_mv_no_constraints_pf['price']
    ex_ante_mv_no_constraints_nav = ( ex_ante_mv_no_constraints_pf['asset_value'].sum() - ex_ante_mv_no_constraints_pf['trading_cost'].sum() )
    budget = ( ex_ante_mv_no_constraints_pf['asset_value'].abs().sum() - ex_ante_mv_no_constraints_pf['trading_cost'].sum() )
    print(f"Unconstrainted MV Portfolio Budget: {budget:.4E}")
    rebalance_mv_no_constraints_budget.append(budget)
    
    ex_post_mv_no_constraints_pf = ex_ante_mv_no_constraints_pf.drop(['shares_diff', 'trading_cost'], axis=1)
    ex_post_mv_no_constraints_pf['weights'] = ef_no_constraints_weights
    ex_post_mv_no_constraints_pf['price'] = prices.iloc[-1].values
    
    conditions = [
                  ex_post_mv_no_constraints_pf['weights'] > 0,
                  ex_post_mv_no_constraints_pf['weights'] < 0
    ]
    values = [1, -1]

    ex_post_mv_no_constraints_pf['long'] = np.select(conditions, values, default=0)
    ex_post_mv_no_constraints_pf['shares'] = allocate(budget, ef_no_constraints_weights, latest_prices)
    ex_post_mv_no_constraints_pf['asset_value'] = ex_post_mv_no_constraints_pf['shares'] * ex_post_mv_no_constraints_pf['price']
    ex_post_mv_no_constraints_pf['time'] = curr_t + relativedelta(days=1)
    #return ex_ante_mv_no_constraints_pf, ex_post_mv_no_constraints_pf

    temp_pf = pd.merge(ex_ante_mv_no_constraints_pf, ex_post_mv_no_constraints_pf, on='ticker', suffixes=('_prev', '_curr'))
    temp_pf['shares_diff'] = temp_pf['shares_curr'] - temp_pf['shares_prev']
    temp_pf['trading_cost'] = 0.2/100 * np.abs(temp_pf['shares_diff']) * temp_pf['price_curr']
    
    # calculate market impact
    price_start = temp_pf.set_index('ticker')['price_curr'].copy()
    
    time_prev_idx = list(temp_pf.columns).index('time_prev')
    temp_prev = temp_pf.iloc[:,0:time_prev_idx+1]
    temp_prev.columns = temp_prev.columns.str.replace(r'_prev$', '', regex=True)
    price_curr_index = list(temp_pf.columns).index('price_curr')
    temp_curr = temp_pf.iloc[:,np.r_[0, price_curr_index:price_curr_index+6]]
    temp_curr.columns = temp_curr.columns.str.replace(r'_curr$', '', regex=True)
    temp_curr
    
    marketimpact = market_impact(daily, sep, msci, crsp, list_of_tickers, temp_curr, temp_prev, curr_t)
    marketimpact_scores = marketimpact.sort_index().values.reshape(-1,1)
    marketimpact.name = 'marketimpact'
    
    price_ = pd.merge(price_start, marketimpact, left_index=True, right_index=True)
    price_['price_exec'] = (1 + price_['marketimpact']/100) * price_['price_curr'] # execution price
    price_['price_diff'] =  price_['price_exec'] - price_['price_curr']
    
    temp_ = temp_pf[['ticker', 'shares_diff', 'trading_cost']].copy()
    temp_ = pd.merge(temp_, price_, left_on='ticker', right_index=True)
    temp_['slippage'] = np.abs(temp_['shares_diff'] * temp_['price_diff'])
    temp_pf['trading_cost'] = temp_pf['trading_cost'].values + temp_['slippage'].values

    temp_pf_nav = temp_pf['asset_value_curr'].sum()    
    #return temp_pf
    
    ex_post_mv_no_constraints_gross_nav = temp_pf['asset_value_curr'].sum()
    mv_no_constraints_gross_nav.append(ex_post_mv_no_constraints_gross_nav)
    rebalance_mv_no_constraints_gross_nav.append(ex_post_mv_no_constraints_gross_nav)
    print(f"Unconstrainted MV Portfolio Gross NAV: {ex_post_mv_no_constraints_gross_nav:.4E}")

    ex_post_mv_no_constraints_net_nav = ( temp_pf['asset_value_curr'].sum() - temp_pf['trading_cost'].sum() )
    mv_no_constraints_net_nav.append(ex_post_mv_no_constraints_net_nav)
    rebalance_mv_no_constraints_net_nav.append(ex_post_mv_no_constraints_net_nav)
    print(f"Unconstrainted MV Portfolio Net NAV: {ex_post_mv_no_constraints_net_nav:.4E}")
    
    mv_no_constraints_budget.append(temp_pf['asset_value_curr'].abs().sum() - temp_pf['trading_cost'].sum())

    portfolio_turnover = np.sum( temp_pf['shares_diff'] )
    mv_no_constraints_turnovers.append(portfolio_turnover)
    print(f"Unconstrainted MV Portfolio Turnover: {portfolio_turnover / ex_post_mv_no_constraints_net_nav * 100:.2f}%")
    
    w = ef_no_constraints_weights * actual_mu
    volatility = np.dot( w.T, np.dot(cov_matrix.values, w) ) * 100
    mv_no_constraints_volatility.append(volatility)
    print(f"Unconstrainted MV Portfolio Volatility: {volatility:.2f}%")
    

    shares_diff_idx = list(temp_pf.columns).index('shares_diff')
    mv_no_constraints_pf = temp_pf.iloc[ :,np.r_[0, shares_diff_idx:temp_pf.shape[1]] ]
    mv_no_constraints_pf.columns = mv_no_constraints_pf.columns.str.replace(r'_curr$','',regex=True)
    return mv_no_constraints_pf

In [60]:
def rebalance_constrainted(mv_constraints_pf, mi_constraints_pf, curr_t, method='ledoit_wolf'):
    '''
    Rebalances constrainted Mean-Variance Portfolio 
    and Market Impact Portfolio at curr_t
    
    mv_constraints_pf::pd.DataFrame: temp_mv_constraints_pf
    mi_constraints_pf::pd.DataFrame: temp_mi_constraints_pf
    curr_t::pd.Timestamp: time of rebalancing
    
    curr_t must be q1_2017 onwards
    '''
    prev_period_preds_idx = timestamps.index(curr_t) - 1
    next_period_preds_idx = timestamps.index(curr_t) + 1
    prev_t_str = timestamps_str[prev_period_preds_idx]
    prev_t = timestamps[prev_period_preds_idx]
    next_t_str = timestamps_str[next_period_preds_idx]
    next_t = timestamps[next_period_preds_idx]
    prev_year = curr_t - relativedelta(years=1)
    print(prev_t, '\t', curr_t, '\t', next_t_str)

    predictions = get_dataframe(preds, next_t_str)
    ensemble_mu = predictions.set_index('ticker')['y_pred']
    actual_mu = predictions.set_index('ticker')['y']
    
    list_of_tickers = predictions['ticker'].values
    prices = get_prices(sep, list_of_tickers, prev_t, curr_t)
    if method == 'ledoit_wolf':
        S = CovarianceShrinkage(prices).ledoit_wolf()
    elif method =='sample_cov':
        S = pypfopt.risk_models.sample_cov(prices)
        
    cov_matrix = pypfopt.risk_models.risk_matrix(prices, returns_data=False, method='sample_cov')
    
    latest_prices = prices.iloc[-2].values
    latest_davs = one_year_DAV(sep, list_of_tickers, curr_t)

    
    # Constrainted Mean-Variance
    ex_ante_mv_constraints_pf = temp_mv_constraints_pf.drop(['price_curr'], axis=1)
    ex_ante_mv_constraints_pf.rename(columns={'price_prev':'price'}, inplace=True)
    ex_ante_mv_constraints_pf['price'] = latest_prices
    ex_ante_mv_constraints_pf['time'] = curr_t
    ex_ante_mv_constraints_pf['asset_value'] = ex_ante_mv_constraints_pf['shares'] * ex_ante_mv_constraints_pf['price']
    ex_ante_mv_constraints_nav = ( ex_ante_mv_constraints_pf['asset_value'].sum() - ex_ante_mv_constraints_pf['trading_cost'].sum() )
    budget = ( ex_ante_mv_constraints_pf['asset_value'].abs().sum() - ex_ante_mv_constraints_pf['trading_cost'].sum() )

    print('\n')
    print('-----------------------------------------------------------------------')
    print(f"Constrainted Mean-Variance Optimization for rebalancing at {curr_t}")
    start_time = time.time()
    mv_ef = EfficientFrontier(ensemble_mu, S, weight_bounds=(-0.015,0.015), solver="MOSEK")
    #mv_ef.add_sector_constraints(sector_mapper, sector_lower, sector_upper)
    #mv_ef.add_constraint( lambda w : cp.abs(w) * budget / latest_prices <= trading_limit(latest_davs, 0.1) )
    mv_ef.efficient_risk(0.05)
    mv_ef_weights = clean_weights(mv_ef.weights, cutoff=0)
    mv_constraints_weights.append(mv_ef_weights)
    print(f"Optimization runtime: {time.time() - start_time:.3f} second(s)")

    print(f"Constrainted MV Portfolio Budget: {budget:.4E}")
    rebalance_mv_constraints_budget.append(budget)
          
    ex_post_mv_constraints_pf = ex_ante_mv_constraints_pf.drop(['shares_diff', 'trading_cost'], axis=1)
    ex_post_mv_constraints_pf['weights'] = mv_ef_weights
    ex_post_mv_constraints_pf['price'] = prices.iloc[-1].values
    
    conditions = [
                  ex_post_mv_constraints_pf['weights'] > 0,
                  ex_post_mv_constraints_pf['weights'] < 0
    ]
    values = [1, -1]
    
    ex_post_mv_constraints_pf['long'] = np.select(conditions, values, default=0)
    ex_post_mv_constraints_pf['shares'] = allocate(budget, mv_ef_weights, latest_prices)
    ex_post_mv_constraints_pf['asset_value'] = ex_post_mv_constraints_pf['shares'] * ex_post_mv_constraints_pf['price']
    ex_post_mv_constraints_pf['time'] = curr_t + relativedelta(days=1)
    #return ex_ante_mv_constraints_pf, ex_post_mv_constraints_pf

    temp_pf = pd.merge(ex_ante_mv_constraints_pf, ex_post_mv_constraints_pf, on='ticker', suffixes=('_prev', '_curr'))
    temp_pf['shares_diff'] = temp_pf['shares_curr'] - temp_pf['shares_prev']
    temp_pf['trading_cost'] = 0.2/100 * np.abs(temp_pf['shares_diff']) * temp_pf['price_curr']

    # calculate market impact
    price_start = temp_pf.set_index('ticker')['price_curr'].copy()
    
    time_prev_idx = list(temp_pf.columns).index('time_prev')
    temp_prev = temp_pf.iloc[:,0:time_prev_idx+1]
    temp_prev.columns = temp_prev.columns.str.replace(r'_prev$', '', regex=True)
    price_curr_index = list(temp_pf.columns).index('price_curr')
    temp_curr = temp_pf.iloc[:,np.r_[0, price_curr_index:price_curr_index+6]]
    temp_curr.columns = temp_curr.columns.str.replace(r'_curr$', '', regex=True)
    temp_curr
    
    marketimpact = market_impact(daily, sep, msci, crsp, list_of_tickers, temp_curr, temp_prev, curr_t)
    marketimpact_scores = marketimpact.sort_index().values.reshape(-1,1)
    marketimpact.name = 'marketimpact'
    
    price_ = pd.merge(price_start, marketimpact, left_index=True, right_index=True)
    price_['price_exec'] = (1 + price_['marketimpact']/100) * price_['price_curr'] # execution price
    price_['price_diff'] =  price_['price_exec'] - price_['price_curr']
    
    temp_ = temp_pf[['ticker', 'shares_diff', 'trading_cost']].copy()
    temp_ = pd.merge(temp_, price_, left_on='ticker', right_index=True)
    temp_['slippage'] = np.abs(temp_['shares_diff'] * temp_['price_diff'])
    temp_pf['trading_cost'] = temp_pf['trading_cost'].values + temp_['slippage'].values
    ex_post_mv_constraints_gross_nav = temp_pf['asset_value_curr'].sum()
    mv_constraints_gross_nav.append(ex_post_mv_constraints_gross_nav)
    rebalance_mv_constraints_gross_nav.append(ex_post_mv_constraints_gross_nav)
    print(f"Constrainted MV Portfolio Gross NAV: {ex_post_mv_constraints_gross_nav:.4E}")

    ex_post_mv_constraints_net_nav = ( temp_pf['asset_value_curr'].sum() - temp_pf['trading_cost'].sum() )
    mv_constraints_net_nav.append(ex_post_mv_constraints_net_nav)
    rebalance_mv_constraints_net_nav.append(ex_post_mv_constraints_net_nav)
    print(f"Constrainted MV Portfolio Net NAV: {ex_post_mv_constraints_net_nav:.4E}")
    
    mv_constraints_budget.append(temp_pf['asset_value_curr'].abs().sum() - temp_pf['trading_cost'].sum())

    portfolio_turnover = np.sum( temp_pf['shares_diff'] ) 
    mv_constraints_turnovers.append(portfolio_turnover)
    print(f"Constrainted MV Portfolio Turnover: {portfolio_turnover / ex_post_mv_constraints_net_nav * 100:.2f}%")
    
    w = mv_ef_weights * actual_mu
    volatility = np.dot( w.T, np.dot(cov_matrix.values, w) ) * 100
    mv_constraints_volatility.append(volatility)
    print(f"Constrainted MV Portfolio Volatility: {volatility:.2f}%")

    shares_diff_idx = list(temp_pf.columns).index('shares_diff')
    mv_constraints_pf = temp_pf.iloc[ :,np.r_[0, shares_diff_idx:temp_pf.shape[1]] ]
    mv_constraints_pf.columns = mv_constraints_pf.columns.str.replace(r'_curr$','',regex=True)
    #return mv_constraints_pf
          
    # Market Impact
    ex_ante_mi_constraints_pf = temp_mi_constraints_pf.drop(['price_curr'], axis=1)
    ex_ante_mi_constraints_pf.rename(columns={'price_prev':'price'}, inplace=True)
    ex_ante_mi_constraints_pf['price'] = latest_prices
    ex_ante_mi_constraints_pf['time'] = curr_t
    ex_ante_mi_constraints_pf['asset_value'] = ex_ante_mi_constraints_pf['shares'] * ex_ante_mi_constraints_pf['price']
    ex_ante_mi_constraints_nav = ( ex_ante_mi_constraints_pf['asset_value'].sum() - ex_ante_mi_constraints_pf['trading_cost'].sum() )
    budget = ( ex_ante_mi_constraints_pf['asset_value'].abs().sum() - ex_ante_mi_constraints_pf['trading_cost'].sum() )
   
    time_prev_idx = list(temp_pf.columns).index('time_prev')
    temp_prev = temp_pf.iloc[:,0:time_prev_idx+1]
    temp_prev.columns = temp_prev.columns.str.replace(r'_prev$', '', regex=True)
    
    price_curr_index = list(temp_pf.columns).index('price_curr')
    temp_curr = temp_pf.iloc[:,np.r_[0, price_curr_index:price_curr_index+6]]
    temp_curr.columns = temp_curr.columns.str.replace(r'_curr$', '', regex=True)
    
    marketimpact = market_impact(daily, sep, msci, crsp, list_of_tickers, temp_curr, temp_prev, curr_t)
    marketimpact_scores = marketimpact.sort_index().values.reshape(-1,1)

    # Market Impact
    print('\n')
    print('-----------------------------------------------------------------------')
    print(f"Market Impact Optimization for rebalancing at {curr_t}")
    start_time = time.time()
    mi_ef = EfficientFrontier(ensemble_mu, S, weight_bounds=(-1,1), solver="MOSEK", verbose=False)
    mi_ef.convex_objective(market_impact_objective, score=marketimpact_scores)
    mi_ef.add_constraint( lambda w : TE_constraint(w, mv_ef_weights, S) <= 0.01 )
    mi_ef.add_constraint( lambda w : cp.abs(w) * budget / latest_prices <= trading_limit(latest_davs, 0.1) )
    mi_ef.efficient_risk(0.05)
    mi_ef_weights = clean_weights(mi_ef.weights, cutoff=0)
    mi_constraints_weights.append(mi_ef_weights)
    print(f"Runtime: {time.time() - start_time:.3f} second(s)")
      
    print(f"Market Impact Portfolio Budget: {budget:.4E}")
    rebalance_mi_constraints_budget.append(budget)
          
    ex_post_mi_constraints_pf = ex_ante_mi_constraints_pf.drop(['shares_diff', 'trading_cost'], axis=1)
    ex_post_mi_constraints_pf['weights'] = mi_ef_weights
    ex_post_mi_constraints_pf['price'] = prices.iloc[-1].values
    
    conditions = [
                  ex_post_mi_constraints_pf['weights'] > 0,
                  ex_post_mi_constraints_pf['weights'] < 0
    ]
    values = [1, -1]
    ex_post_mi_constraints_pf['long'] = np.select(conditions, values, default=0)
    ex_post_mi_constraints_pf['shares'] = allocate(budget, mi_ef_weights, latest_prices)
    ex_post_mi_constraints_pf['asset_value'] = ex_post_mi_constraints_pf['shares'] * ex_post_mi_constraints_pf['price']
    ex_post_mi_constraints_pf['time'] = curr_t + relativedelta(days=1)
    #return ex_ante_mi_constraints_pf, ex_post_mi_constraints_pf

    temp_pf = pd.merge(ex_ante_mi_constraints_pf, ex_post_mi_constraints_pf, on='ticker', suffixes=('_prev', '_curr'))
    temp_pf['shares_diff'] = temp_pf['shares_curr'] - temp_pf['shares_prev']
    temp_pf['trading_cost'] = 0.2/100 * np.abs(temp_pf['shares_diff']) * temp_pf['price_curr']
    
    # calculate market impact
    price_start = temp_pf.set_index('ticker')['price_curr'].copy()
    
    time_prev_idx = list(temp_pf.columns).index('time_prev')
    temp_prev = temp_pf.iloc[:,0:time_prev_idx+1]
    temp_prev.columns = temp_prev.columns.str.replace(r'_prev$', '', regex=True)
    price_curr_index = list(temp_pf.columns).index('price_curr')
    temp_curr = temp_pf.iloc[:,np.r_[0, price_curr_index:price_curr_index+6]]
    temp_curr.columns = temp_curr.columns.str.replace(r'_curr$', '', regex=True)
    temp_curr
    
    marketimpact = market_impact(daily, sep, msci, crsp, list_of_tickers, temp_curr, temp_prev, curr_t)
    marketimpact_scores = marketimpact.sort_index().values.reshape(-1,1)
    marketimpact.name = 'marketimpact'
    
    price_ = pd.merge(price_start, marketimpact, left_index=True, right_index=True)
    price_['price_exec'] = (1 + price_['marketimpact']/100) * price_['price_curr'] # execution price
    price_['price_diff'] =  price_['price_exec'] - price_['price_curr']
    
    temp_ = temp_pf[['ticker', 'shares_diff', 'trading_cost']].copy()
    temp_ = pd.merge(temp_, price_, left_on='ticker', right_index=True)
    temp_['slippage'] = np.abs(temp_['shares_diff'] * temp_['price_diff'])
    temp_pf['trading_cost'] = temp_pf['trading_cost'].values + temp_['slippage'].values    
    
    ex_post_mi_constraints_gross_nav = temp_pf['asset_value_curr'].sum() 
    mi_constraints_gross_nav.append(ex_post_mi_constraints_gross_nav)
    rebalance_mi_constraints_gross_nav.append(ex_post_mi_constraints_gross_nav)
    print(f"Market Impact Portfolio Gross NAV: {ex_post_mi_constraints_gross_nav:.4E}")

    ex_post_mi_constraints_net_nav = ( temp_pf['asset_value_curr'].sum() - temp_pf['trading_cost'].sum() )
    mi_constraints_net_nav.append(ex_post_mi_constraints_net_nav)
    rebalance_mi_constraints_net_nav.append(ex_post_mi_constraints_net_nav)
    print(f"Market Impact Portfolio Net NAV: {ex_post_mi_constraints_net_nav:.4E}")
    
    mi_constraints_budget.append(temp_pf['asset_value_curr'].abs().sum() - temp_pf['trading_cost'].sum())
    
    portfolio_turnover = np.sum( temp_pf['shares_diff'] )
    mi_constraints_turnovers.append(portfolio_turnover)
    print(f"Market Impact Portfolio Turnover: {portfolio_turnover / ex_post_mi_constraints_net_nav * 100:.2f}%")
    
    w = mi_ef_weights * actual_mu
    volatility = np.dot( w.T, np.dot(cov_matrix.values, w) ) * 100
    mi_constraints_volatility.append(volatility)
    print(f"Market Impact Portfolio Volatility: {volatility:.2f}%")
    
    shares_diff_idx = list(temp_pf.columns).index('shares_diff')
    mi_constraints_pf = temp_pf.iloc[ :,np.r_[0, shares_diff_idx:temp_pf.shape[1]] ]
    mi_constraints_pf.columns = mi_constraints_pf.columns.str.replace(r'_curr$','',regex=True)
    
    return mv_constraints_pf, mi_constraints_pf

In [61]:
preds = pd.read_csv(source / 'testing_predictions.csv')
preds.head()

,ground_truth,ensemble_prediction,year-quarter,ticker
0,0.108104,0.041631,2017-1,A
1,0.080545,-0.000002,2017-2,A
2,0.035960,0.008540,2017-3,A
3,-0.041697,0.018393,2017-4,A
4,-0.094635,-0.068808,2018-1,A


In [62]:
tickers = get_dataframe(preds)['ticker']
tickers

0         A
1       AAN
2      AAON
3       AAP
4       ABC
       ... 
617     XOM
618    XRAY
619     ZBH
620    ZBRA
621    ZEUS
Name: ticker, Length: 622, dtype: object

In [63]:
sector_mapper = {}
sector_lower = {}
sector_upper = {}

for ticker in tickers:
    mask = sectors['ticker'] == ticker
    sector_mapper[ticker] = sectors[mask]['sector'].to_string(index=False)

for k,v in sector_mapper.items():
    if k not in sector_lower:
        sector_lower[v] = -0.1
    if k not in sector_upper:
        sector_upper[v] = 0.1

In [64]:
q1_2017_preds = get_dataframe(preds, '2017-1')
q2_2017_preds = get_dataframe(preds, '2017-2')
q3_2017_preds = get_dataframe(preds, '2017-3')
q4_2017_preds = get_dataframe(preds, '2017-4')
q1_2018_preds = get_dataframe(preds, '2018-1')
q2_2018_preds = get_dataframe(preds, '2018-2')
q3_2018_preds = get_dataframe(preds, '2018-3')
q4_2018_preds = get_dataframe(preds, '2018-4')
q1_2019_preds = get_dataframe(preds, '2019-1')
q2_2019_preds = get_dataframe(preds, '2019-2')
q3_2019_preds = get_dataframe(preds, '2019-3')

In [65]:
q4_2016 = pd.Timestamp('2016-12-30')
q1_2017 = pd.Timestamp('2017-03-31')
q2_2017 = pd.Timestamp('2017-06-30')
q3_2017 = pd.Timestamp('2017-09-29')
q4_2017 = pd.Timestamp('2017-12-29')
q1_2018 = pd.Timestamp('2018-03-29')
q2_2018 = pd.Timestamp('2018-06-29')
q3_2018 = pd.Timestamp('2018-09-28')
q4_2018 = pd.Timestamp('2018-12-31')
q1_2019 = pd.Timestamp('2019-03-29')
q2_2019 = pd.Timestamp('2019-06-28')
q3_2019 = pd.Timestamp('2019-09-30')

In [66]:
timestamps_str = [
                  '2016-4',
                  '2017-1','2017-2','2017-3','2017-4',
                  '2018-1','2018-2','2018-3','2018-4',
                  '2019-1','2019-2','2019-3'
]
timestamps = [
              q4_2016,
              q1_2017, q2_2017, q3_2017, q4_2017, 
              q1_2018, q2_2018, q3_2018, q4_2018, 
              q1_2019, q2_2019, q3_2019
              ]

# Initialize starting portfolio
Using CAPM to model returns to initialize portfolio prior to start of backtesting period at Q4-2016.

In [67]:
from pypfopt import expected_returns, risk_models
%time q2_to_q3_2016_prices = get_prices(sep, tickers, pd.Timestamp('2016-06-30'), pd.Timestamp('2016-09-30'))
print( f"No. of null-values: {q2_to_q3_2016_prices.isnull().any().sum()}" )
mu = expected_returns.capm_return(q2_to_q3_2016_prices)
S = risk_models.CovarianceShrinkage(q2_to_q3_2016_prices).ledoit_wolf()
print( len(mu), len(S) )

Requesting for prices at time 2016-09-30 00:00:00
Returning ex-ante prices at time 2016-09-30 00:00:00 
from 2016-06-30 00:00:00 to 2016-09-30 00:00:00
Wall time: 4.39 s
No. of null-values: 0
622 622


In [68]:
q2_to_q3_2016_prices.iloc[-1]

ticker
A        47.09
AAN      25.42
AAON     28.82
AAP     149.12
ABC      80.78
         ...  
XOM      87.28
XRAY     59.43
ZBH     130.02
ZBRA     69.61
ZEUS     22.10
Name: 2016-09-30 00:00:00, Length: 622, dtype: float64

In [69]:
start_capital = 1e8 # 100 million
#start_capital = 1e7 # 10 million
cutoff = 0.005 # 0.5%
latest_prices = q2_to_q3_2016_prices.iloc[-2].values
latest_davs = one_year_DAV(sep, tickers, pd.Timestamp('2016-09-30'))

In [70]:
start_time = time.time()
ef_no_constraints = EfficientFrontier(mu, S, weight_bounds=(-1,1), solver="MOSEK")
ef_no_constraints.efficient_risk(0.05)
ef_no_constraints_weights = clean_weights(ef_no_constraints.weights, cutoff=cutoff)
print(f"Runtime: {time.time() - start_time:.3f} second(s)")

Runtime: 1.996 second(s)


In [71]:
start_time = time.time()
ef_constraints = EfficientFrontier(mu, S, weight_bounds=(-0.015,0.015), solver="MOSEK")
ef_constraints.add_sector_constraints(sector_mapper, sector_lower, sector_upper)
ef_constraints.add_constraint( lambda w : cp.abs(w) * start_capital / latest_prices <= trading_limit(latest_davs, 0.1) )
ef_constraints.efficient_risk(0.05)
ef_constraints_weights = clean_weights(ef_constraints.weights, cutoff=cutoff)

print(f"Runtime: {time.time() - start_time:.3f} second(s)")

Runtime: 1.754 second(s)


In [72]:
unconstrainted = ef_no_constraints_weights.copy()
# No. of Long and Short Holdings:
unconstrainted[unconstrainted < 0] = -1
unconstrainted[unconstrainted > 0] = 1

uniques, counts = np.unique(unconstrainted, return_counts=True)
aggregated_positions_dict = dict(zip(uniques, counts))

if 1 in aggregated_positions_dict:
    print(f"No. of Long stocks: {aggregated_positions_dict[1]}")
if -1 in aggregated_positions_dict:
    print(f"No. of Short Stocks: {aggregated_positions_dict[-1]}")
aggregated_positions_dict

No. of Long stocks: 34


{0.0: 588, 1.0: 34}

In [73]:
shares = allocate(start_capital, ef_no_constraints_weights, latest_prices)

In [74]:
shares = shares.copy()
# No. of Long and Short Holdings:
shares[shares < 0] = -1
shares[shares > 0] = 1

uniques, counts = np.unique(shares, return_counts=True)
aggregated_positions_dict = dict(zip(uniques, counts))

if 1 in aggregated_positions_dict:
    print(f"No. of Long stocks: {aggregated_positions_dict[1]}")
if -1 in aggregated_positions_dict:
    print(f"No. of Short Stocks: {aggregated_positions_dict[-1]}")
aggregated_positions_dict

No. of Long stocks: 34


{0.0: 588, 1.0: 34}

In [75]:
mv_no_constraints_init_pf = initial_pf(sep, preds, '2017-1', pd.Timestamp('2016-09-30'), ef_no_constraints_weights)
print( f"Mean Variance No Constraints Portfolio Start NAV: {mv_no_constraints_init_pf['asset_value'].sum():.4E}" )
mv_constraints_init_pf = initial_pf(sep, preds, '2017-1', pd.Timestamp('2015-09-30'), ef_constraints_weights)
print( f"Mean Variance with Constraints Portfolio Start NAV: {mv_constraints_init_pf['asset_value'].sum():.4E}" )
mi_constraints_init_pf = mv_constraints_init_pf.copy()

Mean Variance No Constraints Portfolio Start NAV: 1.0000E+08
Mean Variance with Constraints Portfolio Start NAV: 6.6974E+07


# Backtest
Q4-2016 -> Q1-2017

In [76]:
start = time.time()

In [77]:
days = [q4_2016]
rebalance_dates = []
mv_no_constraints_weights=[]
mv_constraints_weights = []
mi_constraints_weights = []
rebalance_mv_no_constraints_gross_nav = []
rebalance_mv_constraints_gross_nav = []
rebalance_mi_constraints_gross_nav = []
rebalance_mv_no_constraints_net_nav = []
rebalance_mv_constraints_net_nav = []
rebalance_mi_constraints_net_nav = []
rebalance_mv_no_constraints_budget = []
rebalance_mv_constraints_budget = []
rebalance_mi_constraints_budget = []
mv_no_constraints_gross_nav = []
mv_no_constraints_net_nav = []
mv_no_constraints_turnovers = [0]
mv_no_constraints_volatility = [0]
mv_no_constraints_budget = []
mv_constraints_gross_nav = []
mv_constraints_net_nav = []
mv_constraints_turnovers = [0]
mv_constraints_volatility = [0]
mv_constraints_budget = []
mi_constraints_gross_nav = []
mi_constraints_net_nav = []
mi_constraints_turnovers = [0]
mi_constraints_volatility = [0]
mi_constraints_budget = []

In [78]:
print(
    mv_no_constraints_init_pf['asset_value'].abs().sum(),
    mv_constraints_init_pf['asset_value'].abs().sum(),
    mi_constraints_init_pf['asset_value'].abs().sum()
)

99999632.424 99994196.477 99994196.477


In [79]:
start_time = time.time()
ensemble_mu, S = get_predictions(q4_2016, '2017-1')
ef_no_constraints = EfficientFrontier(ensemble_mu, S, weight_bounds=(-1,1), solver="MOSEK")
ef_no_constraints.efficient_risk(0.05)
ef_no_constraints_weights = clean_weights(ef_no_constraints.weights, cutoff=0.005)
mv_no_constraints_weights.append(ef_no_constraints_weights)
print(f"Runtime: {time.time() - start_time:.3f} second(s)")
# Initial Unconstrainted Mean Variance Weights
budget = mv_no_constraints_init_pf['asset_value'].abs().sum()
print(f"Budget: {budget:.4E}")
rebalance_mv_no_constraints_budget.append(budget)
start_mv_no_constraints_pf = initial_pf(sep, preds, '2017-1', q4_2016, weights=ef_no_constraints_weights, start_capital=budget)
temp_pf = pd.merge(mv_no_constraints_init_pf, start_mv_no_constraints_pf, on='ticker', suffixes=('_prev', '_curr'))
temp_pf['shares_diff'] = temp_pf['shares_curr'] - temp_pf['shares_prev']
temp_pf['trading_cost'] = 0.2/100 * np.abs(temp_pf['shares_diff']) * temp_pf['price_curr']
temp_pf_gross_nav = temp_pf['asset_value_curr'].sum()
mv_no_constraints_gross_nav.append(temp_pf_gross_nav)
rebalance_mv_no_constraints_gross_nav.append(temp_pf_gross_nav)
print(f"Unconstrainted MV Portfolio Gross NAV: {temp_pf_gross_nav:.4E}")
temp_pf_net_nav = ( temp_pf['asset_value_curr'].sum() - temp_pf['trading_cost'].sum() )
mv_no_constraints_net_nav.append(temp_pf_net_nav)
rebalance_mv_no_constraints_net_nav.append(temp_pf_net_nav)
print(f"Unconstrainted MV Portfolio Net NAV: {temp_pf_net_nav:.4E}")
mv_no_constraints_budget.append(temp_pf['asset_value_curr'].abs().sum() - temp_pf['trading_cost'].sum())
# Initial Unconstrainted Mean Variance Portfolio
price_curr_idx = list(temp_pf.columns).index('price_curr')
mv_no_constraints_pf = temp_pf.iloc[ :, np.r_[0, price_curr_idx:temp_pf.shape[1]] ]
mv_no_constraints_pf.columns = mv_no_constraints_pf.columns.str.replace(r'_curr$', '', regex=True)
mv_no_constraints_pf.head()

Requesting for prices at time 2016-12-30 00:00:00
Returning ex-ante prices at time 2016-12-30 00:00:00 
from 2016-09-30 00:00:00 to 2016-12-30 00:00:00
Runtime: 19.591 second(s)
Budget: 1.0000E+08
Unconstrainted MV Portfolio Gross NAV: 2.7961E+07
Unconstrainted MV Portfolio Net NAV: 2.7573E+07


,ticker,price,weights,long,shares,asset_value,time,shares_diff,trading_cost
0,A,45.56,0.00000,0,0.0,0.0,2016-12-30,0.0,0.000
1,AAN,31.99,0.00000,0,0.0,0.0,2016-12-30,0.0,0.000
2,AAON,33.05,0.00000,0,0.0,0.0,2016-12-30,0.0,0.000
3,AAP,169.12,-0.01037,-1,-1850.0,-312872.0,2016-12-30,-1850.0,625.744
4,ABC,78.19,0.00000,0,0.0,0.0,2016-12-30,0.0,0.000


In [80]:
# Preprocessing
temp_prices = get_prices(sep, tickers, pd.Timestamp('2015-12-30'), q4_2016)
latest_prices = temp_prices.iloc[-2].values
latest_davs = one_year_DAV(sep, tickers, q4_2016)
budget = mv_constraints_init_pf['asset_value'].abs().sum()
print(f"MV Constrainted Portfolio Budget: {budget:.4E}")
rebalance_mv_constraints_budget.append(budget)
# Initial Constrainted Mean Variance Weights
start_time = time.time()
ef = EfficientFrontier(ensemble_mu, S, weight_bounds=(-0.015,0.015), solver="MOSEK")
ef.add_sector_constraints(sector_mapper, sector_lower, sector_upper)
ef.add_constraint( lambda w : cp.abs(w) * budget / latest_prices <= trading_limit(latest_davs, 0.1) )
ef.efficient_risk(0.05)
ef_weights = clean_weights(ef.weights, cutoff=0.005)
mv_constraints_weights.append(ef_weights)
print(f"Runtime: {time.time() - start_time:.3f} second(s)")
# Initial Constrainted Mean Variance Portfolio
start_mv_constraints_pf = initial_pf(sep, preds, '2017-1', q4_2016, weights=ef_weights, start_capital=budget)
temp_pf = pd.merge(mv_constraints_init_pf, start_mv_constraints_pf, on='ticker', suffixes=('_prev', '_curr'))
temp_pf['shares_diff'] = temp_pf['shares_curr'] - temp_pf['shares_prev']
temp_pf['trading_cost'] = 0.2/100 * np.abs(temp_pf['shares_diff']) * temp_pf['price_curr']
temp_pf_gross_nav = temp_pf['asset_value_curr'].sum()
print(f"MV Portfolio with Constraints NAV before Trading Cost: {temp_pf_gross_nav:.4E}")
mv_constraints_gross_nav.append(temp_pf_gross_nav)
rebalance_mv_constraints_gross_nav.append(temp_pf_gross_nav)
temp_pf_net_nav = ( temp_pf['asset_value_curr'].sum() - temp_pf['trading_cost'].sum() )
print(f"MV Portfolio with Constraints NAV after Trading Cost: {temp_pf_net_nav:.4E}")
mv_constraints_net_nav.append(temp_pf_net_nav)
rebalance_mv_constraints_net_nav.append(temp_pf_net_nav)
mv_constraints_budget.append(temp_pf['asset_value_curr'].abs().sum() - temp_pf['trading_cost'].sum())
price_curr_idx = list(temp_pf.columns).index('price_curr')
mv_constraints_pf = temp_pf.iloc[ :, np.r_[0, price_curr_idx:temp_pf.shape[1]] ]
mv_constraints_pf.columns = mv_constraints_pf.columns.str.replace(r'_curr$', '', regex=True)
mv_constraints_pf.head()

Requesting for prices at time 2016-12-30 00:00:00
Returning ex-ante prices at time 2016-12-30 00:00:00 
from 2015-12-30 00:00:00 to 2016-12-30 00:00:00
MV Constrainted Portfolio Budget: 9.9994E+07
Runtime: 5.315 second(s)
MV Portfolio with Constraints NAV before Trading Cost: 2.6134E+07
MV Portfolio with Constraints NAV after Trading Cost: 2.5803E+07


,ticker,price,weights,long,shares,asset_value,time,shares_diff,trading_cost
0,A,45.56,0.00000,0,0.0,0.00,2016-12-30,0.0,0.00000
1,AAN,31.99,0.00770,1,6961.0,222682.39,2016-12-30,6961.0,445.36478
2,AAON,33.05,0.00506,1,4428.0,146345.40,2016-12-30,4428.0,292.69080
3,AAP,169.12,-0.01336,-1,-2285.0,-386439.20,2016-12-30,-2285.0,772.87840
4,ABC,78.19,0.00000,0,0.0,0.00,2016-12-30,0.0,0.00000


In [81]:
# Market Impact Preprocessing 
temp_prev = temp_pf.iloc[:,0:7]
temp_prev.columns = temp_prev.columns.str.replace(r'_prev$', '', regex=True)
price_curr_index = list(temp_pf.columns).index('price_curr')
temp_curr = temp_pf.iloc[:,np.r_[0, price_curr_index:price_curr_index+6]]
temp_curr.columns = temp_curr.columns.str.replace(r'_curr$', '', regex=True)
# Market Impact
marketimpact = market_impact(daily, sep, msci, crsp, tickers, temp_curr, temp_prev, q4_2016)
marketimpact_scores = marketimpact.sort_index().values.reshape(-1,1)
# Initial Market Impact Weights
start_time = time.time()
mi_ef = EfficientFrontier(ensemble_mu, S, weight_bounds=(-0.015,0.015), solver="MOSEK", verbose=False)
mi_ef.convex_objective(market_impact_objective, score=marketimpact_scores)
mi_ef.add_constraint( lambda w : TE_constraint(w, ef_weights, S) <= 0.01)
mi_ef.add_constraint( lambda w : cp.abs(w) * budget / latest_prices <= trading_limit(latest_davs, 0.1) )
mi_ef.efficient_risk(0.05)
mi_ef_weights = clean_weights(mi_ef.weights, cutoff=0.005)
mi_constraints_weights.append(mi_ef_weights)
print(f"Runtime: {time.time() - start_time:.3f} second(s)")
# Market Impact Portfolio
print(f"MI Portfolio Budget: {budget:.4E}")
rebalance_mi_constraints_budget.append(budget)
start_mi_constraints_pf = initial_pf(sep, preds, '2017-1', q4_2016, weights=mi_ef_weights, start_capital=budget)
temp_pf = pd.merge(mi_constraints_init_pf, start_mi_constraints_pf, on='ticker', suffixes=('_prev', '_curr'))
temp_pf['shares_diff'] = temp_pf['shares_curr'] - temp_pf['shares_prev']
temp_pf['trading_cost'] = 0.2/100 * np.abs(temp_pf['shares_diff']) * temp_pf['price_curr']
temp_pf_gross_nav = temp_pf['asset_value_curr'].sum()
print(f"MI Portfolio with Constraints NAV before Trading Cost: {temp_pf_gross_nav:.4E}")
mi_constraints_gross_nav.append(temp_pf_gross_nav)
rebalance_mi_constraints_gross_nav.append(temp_pf_gross_nav)
temp_pf_net_nav = ( temp_pf['asset_value_curr'].sum() - temp_pf['trading_cost'].sum() ) 
print(f"MI Portfolio with Constraints NAV after Trading Cost: {temp_pf_net_nav:.4E}")
mi_constraints_net_nav.append(temp_pf_net_nav)
rebalance_mi_constraints_net_nav.append(temp_pf_net_nav)
mi_constraints_budget.append(temp_pf['asset_value_curr'].abs().sum() - temp_pf['trading_cost'].sum())
start_mv_pf = temp_pf.copy()
# Initial Market Impact Portfolio
price_curr_idx = list(temp_pf.columns).index('price_curr')
mi_constraints_pf = temp_pf.iloc[ :, np.r_[0, price_curr_idx:temp_pf.shape[1]] ]
mi_constraints_pf.columns = mi_constraints_pf.columns.str.replace(r'_curr$', '', regex=True)
mi_constraints_pf.head()


New prev_period 2015-12-30 00:00:00 t 2016-12-30 00:00:00
Runtime: 2.662 second(s)
MI Portfolio Budget: 9.9994E+07
MI Portfolio with Constraints NAV before Trading Cost: 2.6712E+07
MI Portfolio with Constraints NAV after Trading Cost: 2.6368E+07


,ticker,price,weights,long,shares,asset_value,time,shares_diff,trading_cost
0,A,45.56,0.00000,0,0.0,0.00,2016-12-30,0.0,0.00000
1,AAN,31.99,0.00759,1,6937.0,221914.63,2016-12-30,6937.0,443.82926
2,AAON,33.05,0.00000,0,0.0,0.00,2016-12-30,0.0,0.00000
3,AAP,169.12,-0.01146,-1,-1982.0,-335195.84,2016-12-30,-1982.0,670.39168
4,ABC,78.19,0.00000,0,0.0,0.00,2016-12-30,0.0,0.00000


In [82]:
print(len(days),
      len(mv_no_constraints_weights), len(mv_constraints_weights),
      len(mi_constraints_weights), len(mv_no_constraints_gross_nav), 
      len(mv_no_constraints_net_nav), len(mv_constraints_gross_nav), 
      len(mv_constraints_net_nav), len(mi_constraints_gross_nav), 
      len(mi_constraints_net_nav), len(mv_no_constraints_budget), 
      len(mv_constraints_budget), len(mi_constraints_budget)
     )

1 1 1 1 1 1 1 1 1 1 1 1 1


In [83]:
start_time = time.time()
for date in pd.date_range(start=q4_2016+relativedelta(days=1), end=q3_2019, freq='D', closed='left'):       
    days.append(date)
    mask = sep['date'] == date
    if sep[mask].shape[0] == 0:
        #print('No data available (non-trading day)', time)
        # append the last NAV
        mv_no_constraints_gross_nav.append(mv_no_constraints_gross_nav[-1])
        mv_no_constraints_net_nav.append(mv_no_constraints_net_nav[-1])
        mv_no_constraints_budget.append(mv_no_constraints_budget[-1])

        mv_constraints_gross_nav.append(mv_constraints_gross_nav[-1])
        mv_constraints_net_nav.append(mv_constraints_net_nav[-1])
        mv_constraints_budget.append(mv_constraints_budget[-1])

        mi_constraints_gross_nav.append(mi_constraints_gross_nav[-1])
        mi_constraints_net_nav.append(mi_constraints_net_nav[-1])
        mi_constraints_budget.append(mi_constraints_budget[-1])
    else:
        # Data available (trading day)
        if date in timestamps: # Rebalance day
            print(f"Rebalancing portfolios at {date}")
            mv_no_constraints_pf = rebalance_unconstrainted(temp_mv_no_constraints_pf, date)
            mv_constraints_pf, mi_constraints_pf = rebalance_constrainted(temp_mv_constraints_pf, temp_mi_constraints_pf, date)
            continue
        # get current prices at date
        temp_sep = sep[mask]
        mask = temp_sep['ticker'].isin(tickers)
        curr_sep = temp_sep[mask].set_index('ticker')['close']
        curr_sep.name = 'price'

        temp_mv_no_constraints_pf = pd.merge(mv_no_constraints_pf, curr_sep, left_on='ticker', right_index=True, suffixes=('_prev', '_curr'))
        if temp_mv_no_constraints_pf.shape[1] > 10:
            price_prev_idx = list(temp_mv_no_constraints_pf.columns).index('price_prev')
            price_curr_idx = list(temp_mv_no_constraints_pf.columns).index('price_curr')
            temp_mv_no_constraints_pf = temp_mv_no_constraints_pf.iloc[:, np.r_[0:3, price_prev_idx, price_curr_idx-5:price_curr_idx+1]].copy()

        temp_mv_no_constraints_pf['asset_value'] = temp_mv_no_constraints_pf['shares'] * temp_mv_no_constraints_pf['price_curr']
        curr_mv_pf_nav = temp_mv_no_constraints_pf['asset_value'].sum()
        
        mv_no_constraints_gross_nav.append(np.round(curr_mv_pf_nav))
        if 'trading_cost' not in temp_mv_no_constraints_pf.columns:
            temp_mv_no_constraints_pf['trading_cost'] = 0.2/100 * np.abs(temp_mv_no_constraints_pf['shares_diff']) * temp_mv_no_constraints_pf['price_curr']
        
        curr_mv_pf_nav -= temp_mv_no_constraints_pf['trading_cost'].sum()
        mv_no_constraints_net_nav.append(np.round(curr_mv_pf_nav))
        
        budget = ( temp_mv_no_constraints_pf['asset_value'].abs().sum() - temp_mv_no_constraints_pf['trading_cost'].sum() )
        mv_no_constraints_budget.append(budget)


        temp_mv_constraints_pf = pd.merge(mv_constraints_pf, curr_sep, left_on='ticker', right_index=True, suffixes=('_prev', '_curr'))
        if temp_mv_constraints_pf.shape[1] > 10:
            price_prev_idx = list(temp_mv_constraints_pf.columns).index('price_prev')
            price_curr_idx = list(temp_mv_constraints_pf.columns).index('price_curr')
            temp_mv_constraints_pf = temp_mv_constraints_pf.iloc[:, np.r_[0:3, price_prev_idx, price_curr_idx-5:price_curr_idx+1]].copy()
            
        temp_mv_constraints_pf['asset_value'] = temp_mv_constraints_pf['shares'] * temp_mv_constraints_pf['price_curr']
        curr_mv_constraints_pf_nav = temp_mv_constraints_pf['asset_value'].sum()
        
        mv_constraints_gross_nav.append(np.round(curr_mv_constraints_pf_nav))
        if 'trading_cost' not in temp_mv_constraints_pf.columns:
            temp_mv_constraints_pf['trading_cost'] = 0.2/100 * np.abs(temp_mv_constraints_pf['shares_diff']) * temp_mv_constraints_pf['price_curr']
        curr_mv_constraints_pf_nav -= temp_mv_constraints_pf['trading_cost'].sum()
        mv_constraints_net_nav.append(np.round(curr_mv_constraints_pf_nav))
        
        budget = ( temp_mv_constraints_pf['asset_value'].abs().sum() - temp_mv_constraints_pf['trading_cost'].sum() )
        mv_constraints_budget.append(budget)

        temp_mi_constraints_pf = pd.merge(mi_constraints_pf, curr_sep, left_on='ticker', right_index=True, suffixes=('_prev', '_curr'))
        if temp_mi_constraints_pf.shape[1] > 10:
            price_prev_idx = list(temp_mi_constraints_pf.columns).index('price_prev')
            price_curr_idx = list(temp_mi_constraints_pf.columns).index('price_curr')
            temp_mi_constraints_pf = temp_mi_constraints_pf.iloc[:, np.r_[0:3, price_prev_idx, price_curr_idx-5:price_curr_idx+1]].copy()
            
        temp_mi_constraints_pf['asset_value'] = temp_mi_constraints_pf['shares'] * temp_mi_constraints_pf['price_curr']
        curr_mi_pf_nav = temp_mi_constraints_pf['asset_value'].sum()
        
        mi_constraints_gross_nav.append(np.round(curr_mi_pf_nav))
        if 'trading_cost' not in temp_mi_constraints_pf.columns:
            temp_mi_constraints_pf['trading_cost'] = 0.2/100 * np.abs(temp_mi_constraints_pf['shares_diff']) * temp_mi_constraints_pf['price_curr']
        curr_mi_pf_nav -= temp_mi_constraints_pf['trading_cost'].sum()
        mi_constraints_net_nav.append(np.round(curr_mi_pf_nav))
        
        budget = ( temp_mi_constraints_pf['asset_value'].abs().sum() - temp_mi_constraints_pf['trading_cost'].sum() )
        mi_constraints_budget.append(budget)

print(f"Finished \n Runtime: {(time.time() - start_time) / 60} minute(s)")

Rebalancing portfolios at 2017-03-31 00:00:00
2016-12-30 00:00:00 	 2017-03-31 00:00:00 	 2017-2
Requesting for prices at time 2017-03-31 00:00:00
Returning ex-ante prices at time 2017-03-31 00:00:00 
from 2016-12-30 00:00:00 to 2017-03-31 00:00:00


-----------------------------------------------------------------------
Unconstrainted Mean-Variance Optimization for rebalancing at 2017-03-31 00:00:00
Optimization runtime: 1.196 second(s)
Unconstrainted MV Portfolio Budget: 1.0447E+08

New prev_period 2016-03-31 00:00:00 t 2017-03-31 00:00:00
Unconstrainted MV Portfolio Gross NAV: 2.0601E+07
Unconstrainted MV Portfolio Net NAV: -7.8284E+06
Unconstrainted MV Portfolio Turnover: 2.93%
Unconstrainted MV Portfolio Volatility: 0.20%
2016-12-30 00:00:00 	 2017-03-31 00:00:00 	 2017-2
Requesting for prices at time 2017-03-31 00:00:00
Returning ex-ante prices at time 2017-03-31 00:00:00 
from 2016-12-30 00:00:00 to 2017-03-31 00:00:00


----------------------------------------------------------

In [84]:
print(len(days),
      len(mv_no_constraints_gross_nav), len(mv_no_constraints_net_nav),
      len(mv_constraints_gross_nav), len(mv_constraints_net_nav),
      len(mi_constraints_gross_nav), len(mi_constraints_net_nav),
      len(mv_no_constraints_budget), len(mv_constraints_budget),
      len(mi_constraints_budget)
     )

1004 1004 1004 1004 1004 1004 1004 1004 1004 1004


In [85]:
results_ = {'date':timestamps[:-1],
            'mv_gross' : rebalance_mv_no_constraints_gross_nav,
            'mv_net' : rebalance_mv_no_constraints_net_nav,
            'mv_budget': rebalance_mv_no_constraints_budget,
            'mv_turnover': mv_no_constraints_turnovers,
            'mv_volatility': mv_no_constraints_volatility,
            'mv_constraints_gross' : rebalance_mv_constraints_gross_nav,
            'mv_constraints_net': rebalance_mv_constraints_net_nav,
            'mv_constraints_budget': rebalance_mv_constraints_budget,
            'mv_constraints_turnover':mv_constraints_turnovers,
            'mv_constraints_volatility': mv_constraints_volatility,
            'mi_constraints_gross': rebalance_mi_constraints_gross_nav,
            'mi_constraints_net': rebalance_mi_constraints_net_nav,
            'mi_constraints_budget': rebalance_mi_constraints_budget,
            'mi_constraints_turnover': mi_constraints_turnovers,
            'mi_constraints_volatility':mi_constraints_volatility
           }
pf_results = pd.DataFrame(results_)
pf_results['date'] = pd.to_datetime(pf_results['date'], format='%Y-%m-%d')
pf_results['mv_turnover'] = pf_results['mv_turnover'] / pf_results['mv_net'] * 100
pf_results['mv_constraints_turnover'] = pf_results['mv_constraints_turnover'] / pf_results['mv_constraints_net'] * 100
pf_results['mi_constraints_turnover'] = pf_results['mi_constraints_turnover'] / pf_results['mi_constraints_net'] * 100
pf_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       11 non-null     datetime64[ns]
 1   mv_gross                   11 non-null     float64       
 2   mv_net                     11 non-null     float64       
 3   mv_budget                  11 non-null     float64       
 4   mv_turnover                11 non-null     float64       
 5   mv_volatility              11 non-null     float64       
 6   mv_constraints_gross       11 non-null     float64       
 7   mv_constraints_net         11 non-null     float64       
 8   mv_constraints_budget      11 non-null     float64       
 9   mv_constraints_turnover    11 non-null     float64       
 10  mv_constraints_volatility  11 non-null     float64       
 11  mi_constraints_gross       11 non-null     float64       
 12  mi_constra

In [86]:
pf_results.to_csv('pf_result-market-impact.csv', index=False)
pf_results.tail()

,date,mv_gross,mv_net,mv_budget,mv_turnover,mv_volatility,mv_constraints_gross,mv_constraints_net,mv_constraints_budget,mv_constraints_turnover,mv_constraints_volatility,mi_constraints_gross,mi_constraints_net,mi_constraints_budget,mi_constraints_turnover,mi_constraints_volatility
6,2018-06-29,6209001.344,1.891223e+06,2.425239e+07,-7.855447,0.049959,8058669.507,1.706953e+06,4.703367e+07,2.642955,0.075456,1.988716e+07,1.243162e+07,8.174371e+07,-0.299309,0.036824
7,2018-09-28,5647280.683,1.613457e+06,2.124251e+07,-7.657968,0.080891,9347465.846,2.820035e+06,4.235543e+07,-2.969856,0.079840,2.129593e+07,1.412720e+07,7.934210e+07,-0.377088,0.069533
8,2018-12-31,2389737.010,-6.161266e+05,1.336602e+07,20.141639,139.148374,3871969.825,-6.723888e+05,2.842091e+07,21.702323,3.768571,1.005542e+07,5.611202e+06,5.780574e+07,-5.749285,6.567150
9,2019-03-29,2368341.519,8.456669e+05,1.161220e+07,5.246983,0.278053,4847496.733,2.236108e+06,2.737374e+07,1.305840,0.295561,1.342153e+07,8.855053e+06,6.064918e+07,0.047566,0.197879
10,2019-06-28,2010541.784,8.164590e+05,1.003921e+07,1.189405,0.495072,3777538.944,1.132176e+06,2.480450e+07,-0.334047,0.187257,1.118711e+07,6.313214e+06,5.693817e+07,1.043827,0.145517


In [87]:
print(len(days),
      len(mv_no_constraints_gross_nav), len(mv_no_constraints_net_nav),
      len(mv_constraints_gross_nav), len(mv_constraints_net_nav),
      len(mi_constraints_gross_nav), len(mi_constraints_net_nav),
      len(mv_no_constraints_budget), len(mv_constraints_budget),
      len(mi_constraints_budget)
     )

1004 1004 1004 1004 1004 1004 1004 1004 1004 1004


In [88]:
daily_ = {
    'date': days,
    'mv_gross_nav' : mv_no_constraints_gross_nav,
    'mv_net_nav' : mv_no_constraints_net_nav,
    'mv': mv_no_constraints_budget,
    'mv_constraints_gross' : mv_constraints_gross_nav,
    'mv_constraints_net': mv_constraints_net_nav,
    'mv_': mv_constraints_budget,
    'mi_constraints_gross': mi_constraints_gross_nav,
    'mi_constraints_net': mi_constraints_net_nav,
    'mi': mi_constraints_budget
           }
pf_daily = pd.DataFrame(daily_)
pf_daily['date'] = pd.to_datetime(pf_daily['date'], format='%Y-%m-%d')
pf_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1004 entries, 0 to 1003
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  1004 non-null   datetime64[ns]
 1   mv_gross_nav          1004 non-null   float64       
 2   mv_net_nav            1004 non-null   float64       
 3   mv                    1004 non-null   float64       
 4   mv_constraints_gross  1004 non-null   float64       
 5   mv_constraints_net    1004 non-null   float64       
 6   mv_                   1004 non-null   float64       
 7   mi_constraints_gross  1004 non-null   float64       
 8   mi_constraints_net    1004 non-null   float64       
 9   mi                    1004 non-null   float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 78.6 KB


In [89]:
pf_daily.to_csv('pf-daily_result-market-impact.csv', index=False)
pf_daily.tail()

,date,mv_gross_nav,mv_net_nav,mv,mv_constraints_gross,mv_constraints_net,mv_,mi_constraints_gross,mi_constraints_net,mi
999,2019-09-25,2328423.0,1134340.0,8.968648e+06,4225100.0,1579737.0,2.267985e+07,12043260.0,7169368.0,5.321854e+07
1000,2019-09-26,2338594.0,1144512.0,8.877743e+06,4254542.0,1609180.0,2.247956e+07,12125570.0,7251677.0,5.276123e+07
1001,2019-09-27,2319722.0,1125639.0,8.834724e+06,4230741.0,1585378.0,2.237845e+07,12120706.0,7246813.0,5.251724e+07
1002,2019-09-28,2319722.0,1125639.0,8.834724e+06,4230741.0,1585378.0,2.237845e+07,12120706.0,7246813.0,5.251724e+07
1003,2019-09-29,2319722.0,1125639.0,8.834724e+06,4230741.0,1585378.0,2.237845e+07,12120706.0,7246813.0,5.251724e+07
